In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [25]:
spark = SparkSession.builder.appName('pyspark_course').getOrCreate()

In [26]:
df_pyspark = spark.read.csv('tips.csv', header=True, inferSchema=True)

In [27]:
df_pyspark.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [28]:
df_pyspark.createOrReplaceGlobalTempView("hotel_data")

In [31]:
sql1 = "select sex, count(*) as cnt from global_temp.hotel_data group by sex"
spark.sql(sql1).show()

+------+---+
|   sex|cnt|
+------+---+
|Female| 87|
|  Male|157|
+------+---+



In [32]:
df_pyspark.groupBy("sex").sum().show()

+------+------------------+-----------------+---------+
|   sex|   sum(total_bill)|         sum(tip)|sum(size)|
+------+------------------+-----------------+---------+
|Female|1570.9499999999998|           246.51|      214|
|  Male|3256.8200000000024|485.0700000000001|      413|
+------+------------------+-----------------+---------+



In [33]:
df_pyspark.groupBy("sex").sum("tip").show()

+------+-----------------+
|   sex|         sum(tip)|
+------+-----------------+
|Female|           246.51|
|  Male|485.0700000000001|
+------+-----------------+



In [35]:
df_pyspark.groupBy("sex").count().show()

+------+-----+
|   sex|count|
+------+-----+
|Female|   87|
|  Male|  157|
+------+-----+



In [38]:
df_pyspark.select("tip").show(7)

+----+
| tip|
+----+
|1.01|
|1.66|
| 3.5|
|3.31|
|3.61|
|4.71|
| 2.0|
+----+
only showing top 7 rows



In [66]:
#sql2 = "select distinct day from global_temp.hotel_data"
sql2 = "select sex, sum(tip) as salary from global_temp.hotel_data \
where tip>3 group by sex order by sum(tip) desc"
spark.sql(sql2).show()

+------+------------------+
|   sex|            salary|
+------+------------------+
|  Male|290.50000000000006|
|Female|            129.32|
+------+------------------+



In [50]:
sql3 = df_pyspark.where("day is not null").select(expr("day as distinct_day")).distinct()
sql3.show()#ACTIONS ONLY WHEN AN ACTION IS CALLED IT WILL TRIGGER A SPARK JOB AND RETURN RES TO EXECUTOR

+------------+
|distinct_day|
+------------+
|        Thur|
|         Sun|
|         Sat|
|         Fri|
+------------+



In [52]:
df_pyspark.where("tip > 2").show()
#here we used show method at end of transformation, thus it will return a df

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     16.29|3.71|  Male|    No|Sun|Dinner|   3|
|     16.97| 3.5|Female|    No|Sun|Dinner|   3|
|     20.65|3.35|  Male|    No|Sat|Dinner|   3|
|     17.92|4.08|  Male|    No|Sat|Dinner|   2|
|     20.29|2.75|Female|    No|Sat|Dinner|   2|
|     15.77|2.23|Female|    No|Sat|Dinner|   2|
|     39.42|7.58|  Male|    No|Sat|Dinner|   4|
|     19.82|3.18|  Male|    No|Sat|Dinne

In [72]:
df_pyspark.select("sex") \
          .where("tip>3") \
          .groupBy("sex") \
           .count().orderBy("count", ascending=False).show()
#count is an action, but here it acts as transformation
#bcoz we have used count after groupby so it is a trans
#if we use count before groupby then it is action

+------+-----+
|   sex|count|
+------+-----+
|  Male|   66|
|Female|   32|
+------+-----+



In [43]:
df2=df_pyspark.withColumnRenamed("tip", "tips")

In [44]:
df_pyspark.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [45]:
df2.show(2)

+----------+----+------+------+---+------+----+
|total_bill|tips|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
+----------+----+------+------+---+------+----+
only showing top 2 rows

